In [ ]:
import numpy as np
import re
import pandas as pd

In [ ]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from transformers import GPT2Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def cleaning(s):
    s = str(s)
    s = re.sub(r'\s\W', ' ', s)
    s = re.sub(r'\W,\s', ' ', s)
    s = re.sub(r"\d+", "", s)
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[!@#$_]', '', s)
    s = s.replace("co", "")
    s = s.replace("https", "")
    s = s.replace("[\w*", " ")
    return s

from datasets import Dataset
from transformers import GPT2Tokenizer

def preprocess_persona_chat(data):
    """Preprocess the persona-chat dataset to create input-output pairs."""
    inputs = []
    outputs = []

    for idx, row in data.iterrows():
        # Clean persona, dialogue, and reference text
        persona = "Persona: " + " ".join([cleaning(p) for p in row["persona_b"]])
        dialogue = "Dialogue: " + " ".join([cleaning(d) for d in row["dialogue"]])
        reference = cleaning(row["reference"])

        # Combine persona and dialogue as input, with the reference as the output
        input_text = f"{persona}\n{dialogue}\n"
        inputs.append(input_text)
        outputs.append(reference)

    return pd.DataFrame({"input": inputs, "output": outputs})

# Load the dataset
df = pd.read_parquet("hf://datasets/Cynaptics/persona-chat/data/train-00000-of-00001.parquet")
processed_data = preprocess_persona_chat(df)

# Convert to Hugging Face dataset for tokenization
hf_dataset = Dataset.from_pandas(processed_data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import DatasetDict

# Load tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Ensure PAD token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize input (persona + dialogue)
    model_inputs = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Tokenize reference as labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["output"],
            padding="max_length",
            truncation=True,
            max_length=512
        )

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn"]
)

# Load model and apply LoRA
model = GPT2LMHeadModel.from_pretrained(model_name)
model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./lora_persona_gpt2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./lora_persona_gpt2")
tokenizer.save_pretrained("./lora_persona_gpt2")

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-8368ac66f494>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.170200,0.163542
2,0.155200,0.162819


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_dir = '/content/drive/MyDrive/fine_tuned_persona_gpt2'
from transformers import Trainer

# Assume trainer is already defined
trainer.save_model(output_dir)

# Save the tokenizer explicitly
tokenizer.save_pretrained(output_dir)

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_response(prompt, model_path, max_length=100):
    """Generate a response based on the input prompt."""
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)

    # Tokenize input
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).input_ids

    # Generate response
    outputs = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example prompt
prompt = """
Person B has the following Persona information.

Persona of Person B: My name is David and I'm a 35 year old math teacher.
Persona of Person B: I like to hike and spend time in the nature.
Persona of Person B: I'm married with two kids.

Instruct: Person A and Person B are now having a conversation.
Following the conversation below, write a response that Person B would say base on the
above Persona information.
Please carefully consider the flow and context of the conversation below, and use the Person B's Persona information appropriately to generate a response that you think are
the most appropriate replying for Person B.

Persona A: Morning! I think I saw you at the parent meeting, what's your name?

Output:
"""

# Generate response
response = generate_response(prompt, "./lora_persona_gpt2")
print("Generated Response:", response)